In [2]:
import tensorflow as tf
import math

## MNIST 예제 tensorflow로 구현하기

In [11]:
# Dense Class
import tensorflow as tf

# 1. Dense Class
class NaiveDense:
	def __init__(self, input_size, output_size, activation):
		self.activation = activation

		# 랜덤값 초기화된 w_shape 크기 행렬 만들기
		w_shape = (input_size, output_size)
		w_initial_value = tf.random.uniform(w_shape, minval = 0, maxval = 1e-1)
		self.W = tf.Variable(w_initial_value)

		# b는 0으로 초기화
		b_shape = (output_size,)
		b_initial_value = tf.zeros(b_shape)
		self.b = tf.Variable(b_initial_value)

	def __call__(self, inputs): # 정방향 패스 수행
		return self.activation(tf.matmul(inputs, self.W) + self.b)

	@property
	def weights(self):
		return [self.W, self.b]

# Sequental class
class NaiveSequential:
	def __init__(self, layers):
		self.layers = layers

	def __call__(self, inputs):
		x = inputs
		for layer in self.layers:
			x = layer(x)
		return x

	@property
	def weights(self):
		weights = []
		for layer in self.layers:
			weights += layer.weights
		return weights

In [12]:
model = NaiveSequential([
			NaiveDense(input_size = 28 * 28, output_size = 512, activation = tf.nn.relu),
			NaiveDense(input_size = 512, output_size = 10, activation = tf.nn.softmax)
])
assert len(model.weights) == 4

In [13]:
class BatchGenerator:
	def __init__(self, images, labels, batch_size = 128):
		assert len(images) == len(labels)
		self.index = 0
		self.images = images
		self.labels = labels
		self.batch_size = batch_size
		self.num_batches = math.ceil(len(images) / batch_size)

	def next(self):
		images = self.images[self.index : self.index + self.batch_size]
		labels = self.labels[self.index : self.index + self.batch_size]
		self.index += self.batch_size
		return images, labels

In [14]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate = 1e-3)

def update_weights(gradients, weights):
	optimizer.apply_gradients(zip(gradients, weights))

def one_training_step(model, images_batch, labels_batch):
	with tf.GradientTape() as tape: # 정방향 패스 실행
		predictions = model(images_batch)
		per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
		average_loss = tf.reduce_mean(per_sample_losses)
	gradients = tape.gradient(average_loss, model.weights) # 가중치 손실 계산
	update_weights(gradients, model.weights) # 가중치 업데이트
	return average_loss


In [15]:
def fit(model, images, labels, epochs, batch_size = 128):
	for epoch_counter in range(epochs):
		print(epoch_counter)
		batch_generator = BatchGenerator(images, labels)
		for batch_counter in range(batch_generator.num_batches):
			images_batch, labels_batch = batch_generator.next()
			loss = one_training_step(model, images_batch, labels_batch)
			if batch_counter % 100 == 0:
				print(f"{batch_counter}번째 배치 손실 : {loss:.2f}")

In [16]:
# 함수 테스트
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs = 10, batch_size = 128)

0
0번째 배치 손실 : 5.51
100번째 배치 손실 : 2.22
200번째 배치 손실 : 2.21
300번째 배치 손실 : 2.10
400번째 배치 손실 : 2.25
1
0번째 배치 손실 : 1.91
100번째 배치 손실 : 1.86
200번째 배치 손실 : 1.84
300번째 배치 손실 : 1.71
400번째 배치 손실 : 1.84
2
0번째 배치 손실 : 1.58
100번째 배치 손실 : 1.56
200번째 배치 손실 : 1.51
300번째 배치 손실 : 1.42
400번째 배치 손실 : 1.52
3
0번째 배치 손실 : 1.32
100번째 배치 손실 : 1.33
200번째 배치 손실 : 1.24
300번째 배치 손실 : 1.20
400번째 배치 손실 : 1.28
4
0번째 배치 손실 : 1.12
100번째 배치 손실 : 1.15
200번째 배치 손실 : 1.05
300번째 배치 손실 : 1.04
400번째 배치 손실 : 1.11
5
0번째 배치 손실 : 0.97
100번째 배치 손실 : 1.01
200번째 배치 손실 : 0.91
300번째 배치 손실 : 0.92
400번째 배치 손실 : 0.99
6
0번째 배치 손실 : 0.86
100번째 배치 손실 : 0.90
200번째 배치 손실 : 0.80
300번째 배치 손실 : 0.83
400번째 배치 손실 : 0.90
7
0번째 배치 손실 : 0.78
100번째 배치 손실 : 0.82
200번째 배치 손실 : 0.72
300번째 배치 손실 : 0.76
400번째 배치 손실 : 0.83
8
0번째 배치 손실 : 0.72
100번째 배치 손실 : 0.75
200번째 배치 손실 : 0.66
300번째 배치 손실 : 0.71
400번째 배치 손실 : 0.77
9
0번째 배치 손실 : 0.67
100번째 배치 손실 : 0.70
200번째 배치 손실 : 0.61
300번째 배치 손실 : 0.66
400번째 배치 손실 : 0.73


In [18]:
import numpy as np
# 모델 평가
predictions = model(test_images)
predictions = predictions.numpy() # tensor -> array
predicted_labels = np.argmax(predictions, axis = 1)
matches = predicted_labels == test_labels

print(f"정확도 : {matches.mean():.2f}")

정확도 : 0.82
